This is ML
 

In [ ]:
#import dependencies
import pandas as pd
import numpy as np
from datetime import date, datetime
import os, sys 
import io
import ee


In [ ]:
## set up pandas dataframe using set_options to warn when we are working on a copy instead of ooriginal dataframe
pd.set_option('mode.chained_assignment','warn')

In [ ]:
# dataframe will display without being truncated
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 25)
pd.set_option('display.width', 1000)

In [ ]:
# !pip install -U -q PyDrive

In [ ]:
#mounting the google drive to access the files
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# command shows file path
# ! ls

In [ ]:
from google.colab import files

In [ ]:
# filepath
path = "/content/drive/My Drive/people_ppp.csv"




In [ ]:
#read the csv, create dataframe
df = pd.read_csv(path,)


In [ ]:
df.head(5)

In [ ]:
df.info()

In [ ]:
#check for nulls
print(df.isna().sum())


In [ ]:
#lost about 10% of rows
df.shape

In [ ]:
df.info()

In [ ]:
#naics saved should not be a float. Need to convert to string before integer
df['NAICS Code']=df['NAICS Code'].astype(str)

In [ ]:
#strip the decimal and zero
df['NAICS Code'] = df['NAICS Code'].str.rstrip('.0')

In [ ]:
#take the first two letters of the NAICS code column and create category column. 
#category column can be matched to named values. The longer code was impractical
df['NAICS Category'] = df['NAICS Code'].str[:2]
df.info()

In [ ]:
df.sample(7)

In [ ]:
#somewhat forcing the date conversion because there must be some values that are not in date format 
df['Date'] =  pd.to_datetime(df['Date'], errors='coerce')

In [ ]:
#converting to date worked
df.info()

In [ ]:
# df['Jobs Saved'] = df['Jobs Saved'].astype(float)
# df['NAICS Category'] = df['NAICS Category'].astype(float)
df['Jobs Saved'] = pd.to_numeric(df['Jobs Saved'], downcast='float')
df['NAICS Category'] = pd.to_numeric(df['NAICS Category'], downcast='float')

In [ ]:
df.dropna(inplace=True)
df.dtypes

In [ ]:
df.shape

In [ ]:
df['Jobs Saved'].dtype
#result says it is a string 


In [ ]:
df['Jobs Saved'] = df['Jobs Saved'].astype(str)


In [ ]:
df['Jobs Saved'].dtype
#result says it is a string 

In [ ]:
#strip off the decimal point if there is one
df['Jobs Saved'] = df['Jobs Saved'].str.rstrip('.0')
df['Jobs Saved'] = df['Jobs Saved'].astype(int)

In [ ]:
df.sample(7)

In [ ]:
df['State'] = df['State'].str.lstrip(' ', )

In [ ]:
df['State'] = df['State'].str.strip('.', )

In [ ]:
df['State'] = df['State'].str.rstrip('"', )

In [ ]:
df['States'] = df['State'].str[:2]

df.info()


In [ ]:
df['States'] = df['States'].str.lstrip(' ', )

In [ ]:
df.head()

In [ ]:
df.sample(7)

In [ ]:
df = df[df['Jobs Saved'] != '']
# df['NAICS Category'] = df['NAICS Category'].str.lstrip(' ', )

In [ ]:
df.sample(5)

In [ ]:
df.info()

In [ ]:
df['States'].value_counts()

In [ ]:
df['Bank'].value_counts()

In [ ]:
banks=df['Bank'].value_counts()

In [ ]:
banks.plot()

In [ ]:
# df['Bank'].where(df['Bank'].apply(lambda x:(x.value_counts()))>=500, "other")

In [ ]:
cond = df['Bank'].value_counts()
threshold = 500
df['Bank'] = np.where(df['Bank'].isin(cond.index[cond >= threshold ]), df['Bank'], 'miscellaneous')

In [ ]:
df['Bank'].value_counts()

In [ ]:
df = df.groupby('States').filter(lambda x: len(x) >= 5)

In [ ]:
df['Business Type'].value_counts()

In [ ]:
# df['NAICS Category'] = df['NAICS Category'].astype(str)

In [ ]:
df.sample(7)

In [ ]:
df['NAICS Category'].value_counts()

In [ ]:
df= df.groupby('NAICS Category').filter(lambda x: len(x) >= 5)


In [ ]:
df.dropna(inplace=True)
df.head(1)

In [ ]:
df.shape

In [ ]:
df.head(1)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import tree
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()


In [ ]:
data=pd.DataFrame()
for x in ['Gender', 'Race', 'Veteran', 'States', 'Bank', 'Business Type']:
  label_encoder.fit(df[x])
  data[x] = label_encoder.transform(df[x])
data['Jobs Saved'] = df['Jobs Saved']
data['NAICS Category'] = df['NAICS Category']
data['Loan Amount'] = df['Loan Amount']

In [ ]:
data.dtypes

In [ ]:
# data = data.fillna(lambda x: x.median())

In [ ]:
data.fillna(data.mean(),inplace=True)

In [ ]:
data.dtypes

In [ ]:
# np.any(np.isnan(X_train)) #true


In [ ]:
data.isnull().any()

In [ ]:
data.shape

In [ ]:
# data['Jobs Saved'].shape


In [ ]:
data['Loan Amount'].shape

In [ ]:
#split loan amounts off and drop from dataframe for ML
target=data['Loan Amount'] 
target.shape


In [ ]:
data=data.drop(['Loan Amount'],axis=1)
data.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)
rf = RandomForestRegressor(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)

Score with: Gender | Race | Veteran | States | Business Type | Jobs Saved | NAICS | Category = 0.4575692683280293

Score with: Gender | Race | Veteran | States | Business Type | Jobs Saved | NAICS | Bank | Category = 0.45253313672369905

Score with: Gender | Race | Veteran | States |  | Jobs Saved | NAICS | Category = 0.4743840791771587

Score with: Gender | Race | Veteran | States |  | NAICS | Category = 0.16463591101903663

ML suggests that Jobs Saved and Business Type influence the amount of money loaned by the bank. Does not necessarily suggest whether or not one was approved or not.




In [ ]:
#score of forest
print(rf.score(X_test, y_test))



In [ ]:
#zip names and feature importance scores
f_imp=rf.feature_importances_
f_name=data.columns
features=zip(f_name, f_imp)
# print(features)
# features into a datafame
f_df=pd.DataFrame(list(features))
print(f_df)


In [ ]:
# X_train.columns
print(rf.feature_importances_)

In [ ]:
data.to_csv('people_ml.csv', index=0)
!cp people_ml.csv "drive/My Drive/"

In [ ]:
#convert the strings to integers in jobs saved
# df['Jobs Saved'] = pd.to_numeric(df['Jobs Saved'], errors="coerce")
# df['Jobs Saved'] = df['Jobs Saved'].astype(int)

In [ ]:
# df.to_csv('clean_ppp.csv', index=0)
# !cp clean_ppp.csv "drive/My Drive/"
